In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Amplitude Analysis with Python basics
TR-999
^^^
+++
✅&nbsp;[ComPWA/RUB-EP1-AG#93](https://github.com/ComPWA/RUB-EP1-AG/issues/93)
:::
::::

# Amplitude Analysis 101

In [ ]:
%pip install -q "phasespace[fromdecay]"==1.9.0 gdown==4.7.1 iminuit==2.25.2 matplotlib==3.7.3 numpy==1.24.4 particle==0.23.0 scipy==1.10.1 vector==1.1.1.post1

In [ ]:
from __future__ import annotations

import warnings

import gdown
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import vector
from vector.backends.numpy import MomentumNumpy4D

warnings.filterwarnings("ignore")

## Phase space

In [ ]:
filename = gdown.cached_download(
    url="https://indico.ific.uv.es/event/6803/contributions/21220/attachments/11209/15505/Three-particles-flat.dat",
    path="data/Three-particles-flat.dat",
    md5="7624074870c22b57581e5c54a1b93754",
    quiet=True,
    verify=False,
)
imported_phsp = np.loadtxt(filename)
imported_phsp.shape

Calculations with 4-vectors are performed with the [`vector`](https://vector.readthedocs.io/en/latest/usage/intro.html) package:

In [ ]:
n_final_state = 3
pa, p1, p2, p3 = (
    vector.array({
        key: imported_phsp[i::4].T[j] for j, key in enumerate(["E", "px", "py", "pz"])
    })
    for i in range(n_final_state + 1)
)

In [ ]:
p0 = p1 + p2 + p3
pb = p0 - pa

p12 = p1 + p2
p23 = p2 + p3
p31 = p3 + p1

s12 = p12.m2
s23 = p23.m2
s31 = p31.m2

In [ ]:
np.testing.assert_almost_equal(p0.p2.max(), 0)

In [ ]:
def boost(p: MomentumNumpy4D, boost_p: MomentumNumpy4D) -> MomentumNumpy4D:
    return p.boost_beta3(boost_p.to_beta3())


def flip(p: MomentumNumpy4D) -> MomentumNumpy4D:
    return MomentumNumpy4D(
        vector.array({"E": p.e, "px": -p.px, "py": -p.py, "pz": -p.pz})
    )

In [ ]:
p1_rest = boost(p1, flip(p1))
np.testing.assert_almost_equal(p1_rest.p.mean(), 0)
np.testing.assert_almost_equal(p1_rest.e.std(), 0, decimal=6)
np.testing.assert_array_almost_equal(p1_rest.e, p1.m)

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("Dalitz plot – histogram")
ax.hist2d(s12, s23, bins=100, cmin=1)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
fig.tight_layout()
plt.show()

## Angular distributions

Before boosting

<iframe src="https://www.geogebra.org/3d/dgjn83pb?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

After Boosted into system$_{12}$ rest frame

<iframe src="https://www.geogebra.org/3d/tv5kr8pp?embed" width="800" height="600" allowfullscreen style="border: 1px solid #e4e4e4;border-radius: 4px;" frameborder="0"></iframe>

The helicity angle between decay product before boost:

The helicity angles $\Omega_i$ are pairs of Euler angles $\Omega_i = \left(\theta_i, \phi_i\right)$:
- Opening angle $\theta_1 \equiv \theta^{12}_1$ of four-momentum $p_1$ is the angle between $p'_1 \equiv p^{12}_1$ and $p_{12}$.
- The angle $\phi_1 \equiv \phi^{12}_1$ defines the angle between the **production plane** spanned by $p_{12}$ and $p_3$ and the **decay plane** spanned by $p^{(\prime)}_1$ and $p^{(\prime)}_2$.

The helicity angles of the other subsystems are defined by cyclic permutation of the indices, such that $\theta_2 \equiv \theta^{12}_2$ and $\theta_3 \equiv \theta^{31}_1$.

In [ ]:
np.testing.assert_array_almost_equal(p1.theta, np.arccos(p1.z / p1.p))
np.testing.assert_array_almost_equal(p1.phi, np.arctan2(p1.y, p1.x))

In [ ]:
theta1_test = p1.rotateZ(-p12.phi).rotateY(-p12.theta).boostZ(p12.beta).theta
theta2_test = p2.rotateZ(-p23.phi).rotateY(-p23.theta).boostZ(p23.beta).theta
theta3_test = p3.rotateZ(-p31.phi).rotateY(-p31.theta).boostZ(p31.beta).theta

phi1_test = p1.rotateZ(-p12.phi).rotateY(-p12.theta).boostZ(p12.beta).phi
phi2_test = p2.rotateZ(-p23.phi).rotateY(-p23.theta).boostZ(p23.beta).phi
phi3_test = p3.rotateZ(-p31.phi).rotateY(-p31.theta).boostZ(p31.beta).phi

In [ ]:
def theta_helicity(p_i: MomentumNumpy4D, p_ij: MomentumNumpy4D):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(p_ij.beta).theta


def phi_helicity(p_i, p_ij):
    return p_i.rotateZ(-p_ij.phi).rotateY(-p_ij.theta).boostZ(p_ij.beta).phi

In [ ]:
theta1 = theta_helicity(p1, p12)
theta2 = theta_helicity(p2, p23)
theta3 = theta_helicity(p3, p31)

phi1 = phi_helicity(p1, p12)
phi2 = phi_helicity(p2, p23)
phi3 = phi_helicity(p3, p31)

In [ ]:
np.testing.assert_array_almost_equal(theta1, theta1_test)
np.testing.assert_array_almost_equal(theta2, theta2_test)
np.testing.assert_array_almost_equal(theta3, theta3_test)

np.testing.assert_array_almost_equal(phi1, phi1_test)
np.testing.assert_array_almost_equal(phi2, phi2_test)
np.testing.assert_array_almost_equal(phi3, phi3_test)

In [ ]:
def plot_helicity_angles_2d(
    phi1, phi2, phi3, theta1, theta2, theta3, title: str
) -> None:
    fig, axes = plt.subplots(figsize=(13, 4), ncols=3, sharey=True)
    for i, ax in enumerate(axes, 1):
        ax.set_xlabel(Rf"$\theta_{i}$")
        ax.set_ylabel(Rf"$\phi_{i}$")
    axes[0].hist2d(theta1[~np.isnan(phi1)], phi1[~np.isnan(phi1)], bins=100)
    axes[1].hist2d(theta2[~np.isnan(phi2)], phi2[~np.isnan(phi2)], bins=100)
    axes[2].hist2d(theta3[~np.isnan(phi3)], phi3[~np.isnan(phi3)], bins=100)
    fig.suptitle(title)
    fig.tight_layout()
    plt.show()


plot_helicity_angles_2d(phi1, phi2, phi3, theta1, theta2, theta3, title="Phase space")

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1,
    bins=100,
    label="phsp",
)
theta_ax[1].hist(
    theta2,
    bins=100,
    label="phsp",
)
theta_ax[2].hist(
    theta3,
    bins=100,
    label="phsp",
)

phi_ax[0].hist(
    phi1,
    bins=50,
    label="phsp",
)
phi_ax[1].hist(
    phi2,
    bins=50,
    label="phsp",
)
phi_ax[2].hist(
    phi3,
    bins=50,
    label="phsp",
)

mass_ax[0].hist(
    p12.m,
    bins=100,
    label="phsp",
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    label="phsp",
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    label="phsp",
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(R"Helicity angles and invariant mass")
fig.tight_layout()
plt.show()

## Amplitude model

```{image} https://github.com/ComPWA/compwa-org/assets/17490173/ec6bf191-bd5f-43b0-a6cb-da470b071630
:width: 100%
```

$$
\begin{eqnarray}
I &=& |A|^2 \\
A &=& A^{12} + A^{23} + A^{31} \\
&1 \equiv \eta ; \quad  2 \equiv \pi^0 ; \quad 3 \equiv p \\
A^{12} &=& \frac{\sum a_m Y_2^m (\Omega_1)}{s-m^2_{a_2}+im_{a_2} \Gamma_{a_2}} \times s^{0.5+0.9u_3} \\
A^{23} &=& \frac{\sum b_m Y_1^m (\Omega_2)}{s-m^2_{\Delta}+im_{\Delta} \Gamma_{\Delta}} \times s^{0.5+0.9t_1} \\
A^{31} &=& \frac{c_0}{s-m^2_{N^*}+im_{N^*} \Gamma_{N^*}} \times s^{1.08+0.2t_2} \\
\end{eqnarray}
$$

where $s, t, u$ are the Mandelstam variables $s_{ij}=(p_i+p_j)^2$, $t_i=(p_a-p_i)^2$, and $u_i=(p_b-p_i)^2$.

In [ ]:
def BW(s, m, Gamma):
    return 1 / (s - m**2 + m * Gamma * 1j)

In [ ]:
u3 = (pb - p3).m2
t1 = (pa - p1).m2
t2 = (pa - p2).m2

The Helicity angle between decay products(Polar Angles):

$$
\theta = \arccos \frac{p_z}{|p|}
$$

In [ ]:
theta_a = np.arccos(pa.pz / pa.p)
theta_1 = np.arccos(p1.pz / p1.p)
theta_2 = np.arccos(p2.pz / p2.p)
theta_3 = np.arccos(p3.pz / p3.p)

The Helicity angle with the production plane and decay plane(Azimuthal Angles):

$$
\phi = \arctan2(p_y , p_x)
$$

In [ ]:
phi_a = np.arctan2(pa.py, pa.px)
phi_1 = np.arctan2(p1.py, p1.px)
phi_2 = np.arctan2(p2.py, p2.px)
phi_3 = np.arctan2(p3.py, p3.px)

$Y_l^m(\phi, \theta)$ is `scipy.special.sph_harm(m, l, phi, theta)`

$Y_l^m(\phi, \theta) = \sqrt{\frac{2n+1}{4\pi}\frac{(n-m)!}{(n+m)!}}e^{im\phi}P_l^m(\cos(\theta))$

here the notation of $\theta$ and $\phi$ are not using the same as in `scipy`

where 
$\phi$ is the azimuthal  from -$\pi$ to $\pi$ (in `scipy` it is $\theta$ and from 0 to $2\pi$)

$\theta$ is the polar angle from 0 to $\pi$ (in `scipy` it is $\phi$)

Spherical harmonics 

In [ ]:
def compute_spherical_harmonics12(theta: np.ndarray, phi: np.ndarray) -> np.ndarray:
    return (
        2.5 * sp.special.sph_harm(2, 2, theta, phi)
        + 4 * sp.special.sph_harm(1, 2, theta, phi)
        + 3.5 * sp.special.sph_harm(0, 2, theta, phi)
        + 0.5 * sp.special.sph_harm(-1, 2, theta, phi)
        + 0 * sp.special.sph_harm(-2, 2, theta, phi)
    )

In [ ]:
PHI, THETA = np.meshgrid(
    np.linspace(-np.pi, +np.pi, num=1_000),
    np.linspace(0, np.pi, num=1_000),
)
Z = compute_spherical_harmonics12(PHI, THETA)

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum a_m Y_2^m (\Omega_1)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum a_m Y_2^m (\Omega_1)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
plt.hist2d(
    p12.phi,
    p12.theta,
    bins=100,
    weights=compute_spherical_harmonics12(p12.phi, p12.theta).real,
    cmap=plt.cm.coolwarm,
)
plt.title("$p_{12}$ with real part of spherical harmonics as weights in histogram")
plt.xlabel(R"$\phi$")
plt.ylabel(R"$\theta$")
plt.show()

In [ ]:
def compute_spherical_harmonics23(theta: np.ndarray, phi: np.ndarray) -> np.ndarray:
    return (
        0.5 * sp.special.sph_harm(1, 1, theta, phi)
        + 4 * sp.special.sph_harm(0, 1, theta, phi)
        - 1.5 * sp.special.sph_harm(-1, 1, theta, phi)
    )

In [ ]:
fig, axes = plt.subplots(figsize=(10, 4), ncols=2, sharey=True, dpi=120)
cmap_real = axes[0].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.real, cmap=plt.cm.coolwarm
)
cmap_imag = axes[1].pcolormesh(
    np.degrees(PHI), np.degrees(THETA), Z.imag, cmap=plt.cm.coolwarm
)

axes[0].set_xlabel(R"$\phi$ [deg]")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[0].set_title(R"Real Part of $\sum b_m Y_1^m (\Omega_2)$")
axes[0].set_ylabel(R"$\theta$ [deg]")
axes[1].set_xlabel(R"$\phi$ [deg]")
axes[1].set_title(R"Imaginary Part of $\sum b_m Y_1^m (\Omega_2)$")

cbar_real = fig.colorbar(cmap_real, ax=axes[0])
cbar_imag = fig.colorbar(cmap_imag, ax=axes[1])

fig.subplots_adjust(wspace=0.4, hspace=0.4)
fig.tight_layout()
plt.rcParams.update({"font.size": 10})
plt.show()

In [ ]:
from scipy.special import lpmv


def wigner_d_function(j, m, n, beta):
    """
    Calculate the Wigner d-function for given j, m, n, and beta.

    Parameters:
    j (int): Total angular momentum quantum number.
    m, n (int): Magnetic quantum numbers.
    beta (float): The angle (in radians).

    Returns:
    float: The value of the Wigner d-function.
    """
    # The Wigner d-function can be related to the associated Legendre polynomials
    # Here, we use scipy's lpmv function which computes the associated Legendre polynomials
    return (
        np.sqrt(
            (np.math.factorial(j + m) * np.math.factorial(j - m))
            / (np.math.factorial(j + n) * np.math.factorial(j - n))
        )
        * np.cos(beta / 2) ** (2 * j + n - m)
        * np.sin(beta / 2) ** (m - n)
        * lpmv(n - m, 2 * j, np.cos(beta))
    )


def spherical_harmonics(l_num, m, theta, phi):
    """
    Calculate the spherical harmonics using the Wigner d-function.

    Parameters:
    l (int): Angular momentum quantum number.
    m (int): Magnetic quantum number.
    theta (float): Polar angle in radians.
    phi (float): Azimuthal angle in radians.

    Returns:
    complex: The value of the spherical harmonic.
    """
    # Calculating the spherical harmonics using Wigner d-function
    # Y^m_l(θ, φ) = √((2l+1)/(4π)) * e^(-imφ) * d^l_{m0}(θ)
    normalization = np.sqrt((2 * l_num + 1) / (4 * np.pi))
    return normalization * np.exp(-1j * m * phi) * wigner_d_function(l_num, m, 0, theta)

In [ ]:
sp.special.sph_harm(1, 1, phi1, theta1)

In [ ]:
spherical_harmonics(1, 1, theta1, phi1)

## Implementation of a kinematic model

In [ ]:
R12 = 1.74
R23 = 1.53
R31 = 2.45
M12 = np.sqrt(R12)
M23 = np.sqrt(R23)
M31 = np.sqrt(R31)

In [ ]:
def BW_model(s12, s23, s31, *, M12, Gamma12, M23, Gamma23, M31, Gamma31):
    A12 = BW(s12, M12, Gamma12)
    A23 = BW(s23, M23, Gamma23)
    A31 = BW(s31, M31, Gamma31)
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
test1 = boost(p1, flip(p1 + p2)).phi
test2 = boost(p1, flip(p1 + p2)).theta

In [ ]:
def SH_model(phi1, theta1, phi2, theta2):
    return (
        np.abs(
            compute_spherical_harmonics12(phi1, theta1)
            + compute_spherical_harmonics23(phi2, theta2)
            + c_0
        )
        ** 2
    )

In [ ]:
def BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
):
    A12 = BW(s12, M12, Gamma12) * compute_spherical_harmonics12(phi1, theta1)
    A23 = BW(s23, M23, Gamma23) * compute_spherical_harmonics23(phi2, theta2)
    A31 = BW(s31, M31, Gamma31) * 1
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
c_0 = 0.25

In [ ]:
def full_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    *,
    M12,
    Gamma12,
    M23,
    Gamma23,
    M31,
    Gamma31,
):
    A12 = (
        BW(s12, M12, Gamma12)
        * compute_spherical_harmonics12(phi1, theta1)
        * s12 ** (0.5 + 0.9 * u3)
    )
    A23 = (
        BW(s23, M23, Gamma23)
        * compute_spherical_harmonics23(phi2, theta2)
        * s23 ** (0.5 + 0.9 * t1)
    )
    A31 = BW(s31, M31, Gamma31) * c_0 * s31 ** (1.08 + 0.2 * t2)
    return np.abs(A12 + A23 + A31) ** 2

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle("For from model: Dalitz Plot of only Breit-Wigner in the formula ")
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=BW_model(
        s12,
        s23,
        s31,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)

fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle("For from model: Dalitz Plots of only spherical harmonics (middle plot)")
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=SH_model(phi1, theta1, phi2, theta2),
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle(
    R"For from model: Dalitz Plots of Breit-Wigner $\times$ Spherical Harmonics"
)
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=BW_SH_model(
        s12,
        s23,
        s31,
        phi1,
        theta1,
        phi2,
        theta2,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-6,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)

fig.tight_layout()
fig.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle("For from model: : Dalitz Plots of full expression of the formula")
hist2 = ax2.hist2d(
    s12,
    s23,
    bins=100,
    weights=full_model(
        s12,
        s23,
        s31,
        phi1,
        theta1,
        phi2,
        theta2,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    ),
    cmin=1e-8,
)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("From model")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("From flat")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
fig.tight_layout()
fig.show()

In [ ]:
weight_BW = BW_model(
    s12,
    s23,
    s31,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)
weight_SH = SH_model(phi1, theta1, phi2, theta2)
weight_BW_SH = BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)
weight_Full = full_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    M12=M12,
    Gamma12=0.1,
    M23=M23,
    Gamma23=0.1,
    M31=M31,
    Gamma31=0.1,
)

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)


phi_ax[0].hist(
    phi1,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)


mass_ax[0].hist(
    p12.m,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    color="red",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_BW_SH,
    color="purple",
    histtype="step",
    label="BW&SH",
    density=True,
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass: phsp and models with good initial guess of parameters"
)
fig.tight_layout()
plt.show()

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_Full,
    label="full",
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_Full,
    label="full",
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_Full,
    label="full",
)

phi_ax[0].hist(
    phi1,
    bins=50,
    weights=weight_Full,
    label="full",
)
phi_ax[1].hist(
    phi2,
    bins=50,
    weights=weight_Full,
    label="full",
)
phi_ax[2].hist(
    phi3,
    bins=50,
    weights=weight_Full,
    label="full",
)

mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_Full,
    label="full",
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_Full,
    label="full",
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_Full,
    label="full",
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass (with the same good initial guess of parameters previously )"
)
fig.tight_layout()
plt.show()

## Data Generation

### Phase space generation

Use of `phasespace`

In [ ]:
import phasespace

pγ_mass = p0.m.mean()
eta_mass = p1.m.mean()
pi_mass = p2.m.mean()
p_mass = p3.m.mean()

weights, particles = phasespace.nbody_decay(
    pγ_mass, [eta_mass, pi_mass, p_mass]
).generate(n_events=100_000)

In [ ]:
import tensorflow as tf


def generate_phsp(
    size: int,
) -> tuple[MomentumNumpy4D, MomentumNumpy4D, MomentumNumpy4D]:
    phsp_sample = generate_phsp_bunch()
    while get_size(phsp_sample) < size:
        bunch = generate_phsp_bunch()
        phsp_sample = concatenate(phsp_sample, bunch)
    phsp_sample = remove_overflow(phsp_sample, size)
    return tuple(to_vector(tensor) for tensor in phsp_sample)


def generate_phsp_bunch() -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    rng = np.random.default_rng(seed=None)
    weights, particles = phasespace.nbody_decay(
        pγ_mass, [eta_mass, pi_mass, p_mass]
    ).generate(n_events=100_000)
    random_weights = rng.uniform(0, weights.numpy().max(), size=weights.shape)
    selector = weights > random_weights
    return tuple(particles[f"p_{i}"][selector] for i in range(n_final_state))


def get_size(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor]):
    return len(phsp[0])


def concatenate(
    phsp1: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
    phsp2: tuple[tf.Tensor, tf.Tensor, tf.Tensor],
) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
    return tuple(tf.concat([phsp1[i], phsp2[i]], axis=0) for i in range(3))


def remove_overflow(phsp: tuple[tf.Tensor, tf.Tensor, tf.Tensor], size: int):
    return tuple(tensor[:size] for tensor in phsp)


def to_vector(tensor: tf.Tensor) -> MomentumNumpy4D:
    return vector.array({
        key: tensor.numpy().T[j] for j, key in enumerate(["px", "py", "pz", "E"])
    })

In [ ]:
p1_phsp, p2_phsp, p3_phsp = generate_phsp(size=1_000_000)

In [ ]:
hist = plt.scatter(
    (p1_phsp + p2_phsp).m2,
    (p2_phsp + p3_phsp).m2,
    s=1e-4,
    c="r",
)
plt.show()

### Hit and miss intensity sample

In [ ]:
def generate_data(model) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    intensities: np.ndarray = model(
        s12,
        s23,
        s31,
        phi1,
        theta1,
        phi2,
        theta2,
        M12=M12,
        Gamma12=0.1,
        M23=M23,
        Gamma23=0.1,
        M31=M31,
        Gamma31=0.1,
    )
    rng = np.random.default_rng(seed=None)
    random_intensities = rng.uniform(0, intensities.max(), size=intensities.shape)
    selector = intensities > random_intensities
    return (
        pa[selector],
        p1[selector],
        p2[selector],
        p3[selector],
    )

In [ ]:
pa_data, p1_data, p2_data, p3_data = generate_data(BW_SH_model)

In [ ]:
p12_data = p1_data + p2_data
p23_data = p2_data + p3_data
p31_data = p3_data + p1_data

s12_data = p12_data.m2
s23_data = p23_data.m2
s31_data = p31_data.m2

In [ ]:
theta1_data = theta_helicity(p1_data, p12_data)
theta2_data = theta_helicity(p2_data, p23_data)
theta3_data = theta_helicity(p3_data, p31_data)
phi1_data = phi_helicity(p1_data, p12_data)
phi2_data = phi_helicity(p2_data, p23_data)
phi3_data = phi_helicity(p3_data, p31_data)

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1_data,
    bins=100,
    label="data",
)
theta_ax[1].hist(
    theta2_data,
    bins=100,
    label="data",
)
theta_ax[2].hist(
    theta3_data,
    bins=100,
    label="data",
)

phi_ax[0].hist(
    phi1_data,
    bins=50,
    label="data",
)
phi_ax[1].hist(
    phi2_data,
    bins=50,
    label="data",
)
phi_ax[2].hist(
    phi3_data,
    bins=50,
    label="data",
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(R"Helicity angles and invariant mass")
fig.tight_layout()
plt.show()

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
theta_ax[0].hist(
    theta1,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


phi_ax[0].hist(
    phi1,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


mass_ax[0].hist(
    p12.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    color="black",
    histtype="step",
    label="phsp",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_BW,
    color="orange",
    histtype="step",
    label="only BW",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_SH,
    color="green",
    histtype="step",
    label="only SH",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)

theta_ax[0].hist(
    theta1_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    theta2_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    theta3_data,
    bins=100,
    label="data",
    density=True,
)

phi_ax[0].hist(
    phi1_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=100,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=100,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)

theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass: phsp, data, and models with good initial guess of parameters"
)
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(12, 5), ncols=2, sharey=True)
fig.suptitle("Dalitz Plots of Phase space and generated data")
hist2 = ax2.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("Generated data")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("Phase space")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")
cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
fig.tight_layout()
fig.show()

## Fitting 

The initial guess of parameters before fittings.
We twitched some parameters in an arbitrary choice values that are not the same as previous sections

In [ ]:
new_weight_BW_SH = BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    M12=M12,
    Gamma12=1,
    M23=M23,
    Gamma23=1,
    M31=M31,
    Gamma31=1,
)

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    theta2_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    theta3_data,
    bins=100,
    label="data",
    density=True,
)

phi_ax[0].hist(
    phi1_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=50,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)


theta_ax[0].hist(
    theta1,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=50,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=new_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass (Before fitting: rough arbitrary guess of parameters for BW $\times$ SH model)"
)
fig.tight_layout()
plt.show()

### Estimator

In [ ]:
def unbinned_nll(M12, Gamma12, M23, Gamma23, M31, Gamma31) -> float:
    phsp = (s12, s23, s31, phi1, theta1, phi2, theta2)
    parameter = dict(
        M12=M12,
        Gamma12=Gamma12,
        M23=M23,
        Gamma23=Gamma23,
        M31=M31,
        Gamma31=Gamma31,
    )
    data = (
        p12_data.m2,
        p23_data.m2,
        p31_data.m2,
        phi1_data,
        theta1_data,
        phi2_data,
        theta2_data,
    )
    model_integral = BW_SH_model(*phsp, **parameter).mean()
    data_intensities = BW_SH_model(*data, **parameter)
    likelihoods = data_intensities / model_integral
    log_likelihood = np.log(likelihoods).sum()
    return -log_likelihood

In [ ]:
%time np.sum(s12)
%time sum(s12)

The unbinned Log likelihood

In [ ]:
unbinned_nll(
    M12=1.32,
    Gamma12=0.1,
    M23=1.24,
    Gamma23=0.1,
    M31=1.57,
    Gamma31=0.1,
)

### Optimizer

In [ ]:
from iminuit import Minuit

initial_parameters = dict(
    M12=1.32,
    Gamma12=0.1,
    M23=1.24,
    Gamma23=0.1,
    M31=1.57,
    Gamma31=0.1,
)
optimizer = Minuit(unbinned_nll, **initial_parameters)
optimizer.errordef = Minuit.LIKELIHOOD
optimizer

In [ ]:
optimizer.migrad()

In [ ]:
optimized_parameters = {p.name: p.value for p in optimizer.params}
optimized_parameters

In [ ]:
fitted_weight_BW_SH = BW_SH_model(
    s12,
    s23,
    s31,
    phi1,
    theta1,
    phi2,
    theta2,
    M12=optimizer.params[0].value,
    Gamma12=optimizer.params[1].value,
    M23=optimizer.params[2].value,
    Gamma23=optimizer.params[3].value,
    M31=optimizer.params[4].value,
    Gamma31=optimizer.params[5].value,
)

In [ ]:
fig, (theta_ax, phi_ax, mass_ax) = plt.subplots(figsize=(13, 11), ncols=3, nrows=3)
for i, ax in enumerate(theta_ax, 1):
    ax.set_title(Rf"$\theta_{i}$")
    ax.set_xticks([0, np.pi / 2, np.pi])
    ax.set_xticklabels(["0", R"$\frac{\pi}{2}$", R"$\pi$"])
for i, ax in enumerate(phi_ax, 1):
    ax.set_title(Rf"$\phi_{i}$")
    ax.set_xticks([-np.pi, 0, np.pi])
    ax.set_xticklabels([R"-$\pi$", 0, R"$\pi$"])
for i, ax in enumerate(mass_ax, 1):
    ax.set_title(Rf"$m_{{{i}{(i % 3 + 1)}}}$")

theta_ax[0].hist(
    theta1_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[1].hist(
    theta2_data,
    bins=100,
    label="data",
    density=True,
)
theta_ax[2].hist(
    theta3_data,
    bins=100,
    label="data",
    density=True,
)

phi_ax[0].hist(
    phi1_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[1].hist(
    phi2_data,
    bins=50,
    label="data",
    density=True,
)
phi_ax[2].hist(
    phi3_data,
    bins=50,
    label="data",
    density=True,
)

mass_ax[0].hist(
    p12_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[1].hist(
    p23_data.m,
    bins=100,
    label="data",
    density=True,
)
mass_ax[2].hist(
    p31_data.m,
    bins=100,
    label="data",
    density=True,
)


theta_ax[0].hist(
    theta1,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[1].hist(
    theta2,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
theta_ax[2].hist(
    theta3,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[0].hist(
    phi1,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[1].hist(
    phi2,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
phi_ax[2].hist(
    phi3,
    bins=50,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[0].hist(
    p12.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[1].hist(
    p23.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)
mass_ax[2].hist(
    p31.m,
    bins=100,
    weights=fitted_weight_BW_SH,
    color="red",
    histtype="step",
    label="BW&SH",
    density=True,
)


theta_ax[0].legend()
phi_ax[0].legend()
theta_ax[1].legend()
phi_ax[1].legend()
theta_ax[2].legend()
phi_ax[2].legend()

mass_ax[0].legend()
mass_ax[1].legend()
mass_ax[2].legend()


fig.suptitle(
    R"Helicity angles and invariant mass (After fitting: updated fitted parameters for BW $\times$ SH model)"
)
fig.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(figsize=(12, 4), ncols=3, sharey=True)
fig.suptitle(
    R"Dalitz Plots of Phase space, generated data, and fitted BW $\times$ SH model"
)
hist2 = ax2.hist2d(s12_data, s23_data, bins=100, cmin=1e-6)
ax2.set_xlabel(R"$s_{12}$")
ax2.set_title("Generated data")
ax2.set_ylabel(R"$s_{23}$")

hist1 = ax1.hist2d(s12, s23, bins=100, cmin=1e-6)
ax1.set_title("Phase space")
ax1.set_xlabel(R"$s_{12}$")
ax1.set_ylabel(R"$s_{23}$")

hist3 = ax3.hist2d(s12, s23, bins=100, weights=fitted_weight_BW_SH, cmin=1e-6)
ax3.set_title(R"BW $\times$ SH model")
ax3.set_xlabel(R"$s_{12}$")
ax3.set_ylabel(R"$s_{23}$")

cbar1 = fig.colorbar(hist1[3], ax=ax1)
cbar2 = fig.colorbar(hist2[3], ax=ax2)
cbar3 = fig.colorbar(hist3[3], ax=ax3)

fig.tight_layout()
fig.show()

In [ ]:
pa

In [ ]:
pb